# Prorrata ERNC
Este programa tiene por objetivo realizar el re-calculo de curtailment para el sistema, en base a metodología propuesta por la NTCyO.

## LECTURA DE DATOS
Los datos deben ser extraidos desde el accdb en potencia neta (no por la potencia, sino por los marginales no truncados). La lista de datos que se deben extraer son:
1. Generación de cada central.
2. Perfil de generación de cada central.
3. Barra asociada a cada central.
4. Costos marginales para cada.
5. Curtailment por central (quizas por barra es suficiente).
6. Potencia máxima.
7. Generación disponible.
8. Estado de operación.

In [1]:
import polars as pl
#import datetime as dt

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
    inspect
)

# fecha_prg = dt.date(
#     year=2023,
#     month=12,
#     day=27
# )
# 
# path_prg = Path(f"R:\{fecha_prg.year}\{fecha_prg:%Y%m}\Prg {fecha_prg:%Y%m%d}\Antecedentes\Model PRGdia_Full_Definitivo Solution.accdb")
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)
    tables = inspect(prg_engine).get_table_names()

    for table in tables:
        print(f"trabajando en tabla: {table}...")
        (
            pl.read_database(query=f"SELECT * FROM {table}", connection=prg_engine)
            .write_parquet(f"../data/tables/{table}.parquet")
        )

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    prg_engine.dispose()

trabajando en tabla: t_attribute...
trabajando en tabla: t_attribute_data...
trabajando en tabla: t_band...
trabajando en tabla: t_category...
trabajando en tabla: t_class...
trabajando en tabla: t_class_group...
trabajando en tabla: t_collection...
trabajando en tabla: t_config...
trabajando en tabla: t_custom_column...
trabajando en tabla: t_data_0...
trabajando en tabla: t_data_1...
trabajando en tabla: t_data_2...
trabajando en tabla: t_data_3...
trabajando en tabla: t_data_4...
trabajando en tabla: t_data_6...
trabajando en tabla: t_data_7...
trabajando en tabla: t_data_current...
trabajando en tabla: t_key...
trabajando en tabla: t_key_index...
trabajando en tabla: t_membership...
trabajando en tabla: t_memo_object...
trabajando en tabla: t_model...
trabajando en tabla: t_object...
trabajando en tabla: t_object_meta...
trabajando en tabla: t_period_0...
trabajando en tabla: t_period_1...
trabajando en tabla: t_period_2...
trabajando en tabla: t_period_3...
trabajando en tabla: t_

In [13]:
data_0 = pl.read_database(query="SELECT * FROM t_data_0", connection=prg_engine)
data_0.head()

key_id,period_id,value
i64,i64,f64
1,1,96.955687
1,2,0.0
1,3,0.0
1,4,0.0
1,5,0.0


In [14]:
prg_engine.connect().close()